In [1]:
# | hide

from IPython.display import Markdown as md
from IPython.display import Code


# faststream_gen

The FastStream-gen library uses advanced AI to generate `FastStream` code from user descriptions, speeding up `FastStream` app development.

---

![PyPI](https://img.shields.io/pypi/v/faststream-gen)
![PyPI - Downloads](https://img.shields.io/pypi/dm/faststream-gen)
![PyPI - Python Version](https://img.shields.io/pypi/pyversions/faststream-gen)

![GitHub Workflow Status](https://img.shields.io/github/actions/workflow/status/airtai/fastkafka-gen/test.yaml)
![GitHub](https://img.shields.io/github/license/airtai/fastkafka-gen)

---

If you've heard about the `FastStream` library and maybe wanted to try it out, but didn't have time to go through the documentation, no problem, `faststream_gen` will generate your `FastStream` application. Also, to make sure that the generated application really works, we will also generate tests for you. All you need to do is provide the desription of the application you want and `faststream_gen` will generate it for you!

## Set up your OPENAI_API_KEY

If you don't already have `OPENAI_API_KEY`, you can create one at [OPENAI API keys](https://platform.openai.com/account/api-keys)

## Install

faststream_gen is published as a Python package and can be installed with pip:

```sh
pip install faststream_gen
```

If the installation was successful, you should now have the faststream_gen installed on your system. Run the below command from the terminal to see the full list of available commands:

```sh
faststream_gen --help
```

In [2]:
# | echo: false
! faststream_gen --help

                                                                                
 Usage: faststream_gen [OPTIONS] [DESCRIPTION]                                  
                                                                                
 Effortlessly generate FastStream application code and integration tests from   
 the app description.                                                           
                                                                                
╭─ Arguments ──────────────────────────────────────────────────────────────────╮
│   description      [DESCRIPTION]  Summarize your FastStream app in a few     │
│                                   sentences!                                 │
│                                                                              │
│                                   Include details about messages, topics,    │
│                                   servers, and a brief overview of the       │
│                           

## How to use

This library is very easy to use. The only thing you need to do is prepare a description of your `FastStream` application, i.e. write what your application should do and pass it to the `faststrem_gen` command as a `string`:

```sh
faststream_gen "your_app_description"
```

### Example

In [3]:
# | hide

with open('../docs_src/app_index_example/app_description.txt', 'r') as file:
    description = file.read().replace('\n', '').replace('"', "'")

faststream_gen_example = f"""
```sh
faststream_gen  "{description}" 
✨  Generating a new FastStream application!
 ✔ Application description validated 
 ✔ FastStream app skeleton generated and saved at: ./faststream-gen/application_skeleton.py 
 ✔ FastStream app generated and saved at: ./faststream-gen/application.py 
 ✔ Tests are generated and saved at: ./faststream-gen/test.py 
 Tokens used: 18572
 Total Cost (USD): $0.05635
✨  All files were successfully generated!
```

**Note**: If you have longer application description (and we encourage writing a detailed description), it is easier to save description into the file and use:
```sh
faststream_gen -i path_to_app_description/description.txt
```
"""

In [4]:
# | echo: false

md(faststream_gen_example)


```sh
faststream_gen  "Write a faststream application with with one consumer function and two producer functions. The consumer function should receive the a message posted on 'new_joinee' topic. The message should contain 'employee_name', 'age', 'location' and 'experience' attributes. After consuming the consumer function should send the details to the 'project_team' and 'admin_team' topics. Use only localhost broker" 
✨  Generating a new FastStream application!
 ✔ Application description validated 
 ✔ FastStream app skeleton generated and saved at: ./faststream-gen/application_skeleton.py 
 ✔ FastStream app generated and saved at: ./faststream-gen/application.py 
 ✔ Tests are generated and saved at: ./faststream-gen/test.py 
 Tokens used: 18572
 Total Cost (USD): $0.05635
✨  All files were successfully generated!
```

**Note**: If you have longer application description (and we encourage writing a detailed description), it is easier to save description into the file and use:
```sh
faststream_gen -i path_to_app_description/description.txt
```


`faststream_gen` CLI will create `faststream-gen` folder and save generated FastStream `application.py` inside the folder. We know that no one likes code full of errors, that's why we generate and run tests along with the generated application! Next to `application.py` you will find the `test.py` script for testing your FastStream application!

In the `faststream-gen` folder you will find following files:

`application.py`

In [5]:
# | echo: false

code = Code(filename='../docs_src/app_index_example/application.py', language='python')
md(f"```python\n{code}\n```")

```python
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Employee(BaseModel):
    employee_name: str = Field(
        ..., examples=["John Doe"], description="Employee name"
    )
    age: int = Field(
        ..., examples=[30], description="Age"
    )
    location: str = Field(
        ..., examples=["New York"], description="Location"
    )
    experience: int = Field(
        ..., examples=[5], description="Experience in years"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.subscriber("new_joinee")
@broker.publisher("project_team")
@broker.publisher("admin_team")
async def on_new_joinee(msg: Employee, logger: Logger):
    logger.info(msg)
    return msg
```

`test.py`

In [6]:
# | echo: false

code = Code(filename='../docs_src/app_index_example/test.py', language='python')
md(f"```python\n{code}\n```")

```python
import pytest

from faststream.kafka import TestKafkaBroker

from application import *


@broker.subscriber("project_team")
async def on_project_team(msg: Employee):
    pass

@broker.subscriber("admin_team")
async def on_admin_team(msg: Employee):
    pass

@pytest.mark.asyncio
async def test_app():
    async with TestKafkaBroker(broker):
        await broker.publish(Employee(employee_name="John Doe", age=30, location="New York", experience=5), "new_joinee")

        on_new_joinee.mock.assert_called_with(dict(Employee(employee_name="John Doe", age=30, location="New York", experience=5)))
        on_project_team.mock.assert_called_with(dict(Employee(employee_name="John Doe", age=30, location="New York", experience=5)))
        on_admin_team.mock.assert_called_with(dict(Employee(employee_name="John Doe", age=30, location="New York", experience=5)))
```

Finally, you can personally check whether all the generated tests pass:

```sh
pytest faststream-gen/test.py
```

## Copyright
Copyright © 2023 onwards airt technologies ltd, Inc.

## License

This project is licensed under the terms of the <a href="https://github.com/airtai/fastkafka-gen/blob/main/LICENSE" target="_blank">Apache License 2.0</a>